In [3]:
import pandas as pd
import numpy as np
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
df = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                   "toy": [np.nan, 'Batmobile', 'Bullwhip'],
                   "born": [pd.NaT, pd.Timestamp("1940-04-25"),
                            pd.NaT]})

In [12]:
df2 = df[['name']]
df2


,name
0,Alfred
1,Batman
2,Catwoman


In [8]:
[i for i in range(1, 6)]

[1, 2, 3, 4, 5]

In [6]:
df['toy'] = df['born']
df

,name,toy,born
0,Alfred,NaT,NaT
1,Batman,1940-04-25,1940-04-25
2,Catwoman,NaT,NaT


In [15]:
a  = [np.array([47.21627161]),
  np.array([46.82207559]),
  np.array([47.52026632]),
  np.array([47.39415012]),
  np.array([47.39021865])]

In [41]:
c  = [np.array([47.21627161,2]),
  np.array([46.82207559,3]),
  np.array([47.52026632,2]),
  np.array([47.39415012,2]),
  np.array([47.39021865,5])]

In [16]:
a


[array([47.21627161]),
 array([46.82207559]),
 array([47.52026632]),
 array([47.39415012]),
 array([47.39021865])]

In [27]:
b = [
  {"a": 1, "b": 2, "c": "string"},
  {"a": 3, "b": 4, "c": "string"},
  {"a": 5, "b": 6, "c": "string"}
]

In [43]:
c

[array([47.21627161,  2.        ]),
 array([46.82207559,  3.        ]),
 array([47.52026632,  2.        ]),
 array([47.39415012,  2.        ]),
 array([47.39021865,  5.        ])]

In [50]:
d = [(1,2), (3,2), (4,5)]
e = [(1,2,'a'), (3,2,'b'), (4,5,'c')]

In [59]:
for idx, i in enumerate(e):
    print(idx, i)

0 (1, 2, 'a')
1 (3, 2, 'b')
2 (4, 5, 'c')


In [68]:
f = [(0.47880892107910455, 'Pearson correlation'), (0.4758693053729348, 'Pearson correlation'), (0.47914140204022543, 'Pearson correlation'), (0.4858238824915101, 'Pearson correlation'), (0.4840002529809448, 'Pearson correlation')]

In [78]:
def avgs(l):
    if isinstance(l[0] , (int, float)):
        return sum(l) / len(l)
    if isinstance(l[0], np.ndarray):
        return np.average(l, axis = 0)        
    if isinstance(l[0], tuple):
        #return np.average(l, axis = 0)
        numeric_averages = {}
        for tuple_obj in l:
            for idx, value in enumerate(tuple_obj):
                if isinstance(value, (int, float)):                    
                    numeric_averages[idx] = numeric_averages.get(idx, 0) + value        
        r = []
        for i in range (len(l[0])):            
            if i in numeric_averages:                
                r.append(numeric_averages[i] / len(l))
            else:
                
                r.append(l[0][i])
        return tuple(r)
    if isinstance(l[0], dict):
        objects = l
        numeric_averages = {}
        for obj in l:
            for key, value in obj.items():
                if isinstance(value, (int, float)):
                    numeric_averages[key] = numeric_averages.get(key, 0) + value
        
        for key, total in numeric_averages.items():
            count = len([obj for obj in objects if isinstance(obj.get(key), (int, float)) or (isinstance(obj, list) or isinstance(obj, np.ndarray) and isinstance(value, (int, float)))])
            numeric_averages[key] = total / count
        return numeric_averages
#avgs([1,2,3])
#avgs(b)
avgs(f)

{0: 2.4036437639647197}
0
1


(0.4807287527929439, 'Pearson correlation')

In [76]:
f[0][1]

'Pearson correlation'

In [19]:
def average_numeric_values(objects):
  """
  Averages numeric values within a list of objects.

  Args:
    objects (list): A list of objects containing numeric values.

  Returns:
    dict: A dictionary where keys are numeric attribute names and values are their averages.
  """

  numeric_averages = {}
  for obj in objects:
    for key, value in obj.items():
      if isinstance(value, (int, float)):
        numeric_averages[key] = numeric_averages.get(key, 0) + value

  for key, total in numeric_averages.items():
    count = len([obj for obj in objects if isinstance(obj.get(key), (int, float))])
    numeric_averages[key] = total / count

  return numeric_averages

# Example usage
objects = [
  {"a": 1, "b": 2, "c": "string"},
  {"a": 3, "b": 4, "c": "string"},
  {"a": 5, "b": 6, "c": "string"}
]

average_results = average_numeric_values(objects)
print("Average results:", average_results)

Average results: {'a': 3.0, 'b': 4.0}


In [20]:
average_numeric_values(a) 

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [21]:
import numpy as np

def average_numeric_values(objects):
  """
  Averages numeric values within a list of objects, handling various data types.

  Args:
    objects (list): A list of objects containing numeric values.

  Returns:
    dict: A dictionary where keys are numeric attribute names and values are their averages.
  """

  numeric_averages = {}
  for obj in objects:
    if isinstance(obj, dict):
      for key, value in obj.items():
        if isinstance(value, (int, float)):
          numeric_averages[key] = numeric_averages.get(key, 0) + value
    elif isinstance(obj, list) or isinstance(obj, np.ndarray):
      for value in obj:
        if isinstance(value, (int, float)):
          numeric_averages["list_element"] = numeric_averages.get("list_element", 0) + value

  for key, total in numeric_averages.items():
    count = len([obj for obj in objects if isinstance(obj.get(key), (int, float)) or (isinstance(obj, list) or isinstance(obj, np.ndarray) and isinstance(value, (int, float)))])
    numeric_averages[key] = total / count

  return numeric_averages

# Example usage
objects = [
  {"a": 1, "b": 2, "c": "string"},
  {"a": 3, "b": 4, "c": "string"},
  {"a": 5, "b": 6, "c": "string"},
  [7, 8, 9],
  np.array([10, 11, 12])
]

average_results = average_numeric_values(objects)
print("Average results:", average_results)

AttributeError: 'list' object has no attribute 'get'

In [79]:
def run(data):
    data = data[['PA195Q01JA', 'STUD_MATH']].dropna()
    
    correlation = data['PA195Q01JA'].corr(data['STUD_MATH'])
    
    return correlation, 'Pearson Correlation'

In [83]:
df_stu, meta_stu = pyreadstat.read_sav('data/CY08MSP_STU_QQQ_HUN.sav')
df_stu['STUD_MATH'] = df_stu['PV1MATH']

In [84]:
run(df_stu)

(nan, 'Pearson Correlation')

In [85]:
data = df_stu


In [86]:
data = data[['PA195Q01JA', 'STUD_MATH']].dropna()

In [87]:
data


,PA195Q01JA,STUD_MATH


In [88]:
df_stu

,CNT,CNTRYID,CNTSCHID,CNTSTUID,CYC,NatCen,STRATUM,SUBNATIO,REGION,OECD,...,PV4MPRE,PV5MPRE,PV6MPRE,PV7MPRE,PV8MPRE,PV9MPRE,PV10MPRE,SENWT,VER_DAT,STUD_MATH
0,HUN,348.0,34800278.0,34800001.0,08MS,034800,HUN05,3480000,34800.0,1.0,...,640.519,611.919,645.604,625.981,598.866,617.845,588.401,0.89359,02MAY23:12:28:56,588.531
1,HUN,348.0,34800070.0,34800002.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,483.582,381.121,470.334,416.643,433.969,480.882,406.840,0.75935,02MAY23:12:28:55,397.170
2,HUN,348.0,34800201.0,34800003.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,618.097,686.393,672.865,690.185,677.799,739.687,701.354,0.76834,02MAY23:12:28:55,614.811
3,HUN,348.0,34800037.0,34800004.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,296.304,312.457,306.082,306.623,257.003,282.050,349.307,0.94003,02MAY23:12:28:56,265.417
4,HUN,348.0,34800120.0,34800006.0,08MS,034800,HUN01,3480000,34800.0,1.0,...,332.972,291.542,364.734,392.895,346.364,300.835,396.804,1.39470,02MAY23:12:28:54,377.679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6193,HUN,348.0,34800274.0,34808759.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,413.443,431.441,429.115,439.152,462.193,486.447,443.367,0.68586,02MAY23:12:28:55,400.780
6194,HUN,348.0,34800003.0,34808760.0,08MS,034800,HUN05,3480000,34800.0,1.0,...,339.807,420.217,381.860,410.052,379.095,405.908,386.596,0.59395,02MAY23:12:28:56,348.603
6195,HUN,348.0,34800209.0,34808761.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,446.405,368.627,441.561,430.339,369.407,452.986,392.082,0.72933,02MAY23:12:28:56,419.138
6196,HUN,348.0,34800125.0,34808764.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,393.713,286.782,303.800,436.778,413.684,368.423,365.617,0.97244,02MAY23:12:28:56,367.845


In [89]:
data = df_stu[['PA195Q01JA', 'STUD_MATH']]

In [90]:
data


,PA195Q01JA,STUD_MATH
0,NaN,588.531
1,NaN,397.170
2,NaN,614.811
3,NaN,265.417
4,NaN,377.679
...,...,...
6193,NaN,400.780
6194,NaN,348.603
6195,NaN,419.138
6196,NaN,367.845


In [91]:
df_stu['PA195Q01JA']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
6193   NaN
6194   NaN
6195   NaN
6196   NaN
6197   NaN
Name: PA195Q01JA, Length: 6198, dtype: float64

In [92]:
df_stu.dropna(axis=1, how='all')


,CNT,CNTRYID,CNTSCHID,CNTSTUID,CYC,NatCen,STRATUM,SUBNATIO,REGION,OECD,...,PV4MPRE,PV5MPRE,PV6MPRE,PV7MPRE,PV8MPRE,PV9MPRE,PV10MPRE,SENWT,VER_DAT,STUD_MATH
0,HUN,348.0,34800278.0,34800001.0,08MS,034800,HUN05,3480000,34800.0,1.0,...,640.519,611.919,645.604,625.981,598.866,617.845,588.401,0.89359,02MAY23:12:28:56,588.531
1,HUN,348.0,34800070.0,34800002.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,483.582,381.121,470.334,416.643,433.969,480.882,406.840,0.75935,02MAY23:12:28:55,397.170
2,HUN,348.0,34800201.0,34800003.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,618.097,686.393,672.865,690.185,677.799,739.687,701.354,0.76834,02MAY23:12:28:55,614.811
3,HUN,348.0,34800037.0,34800004.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,296.304,312.457,306.082,306.623,257.003,282.050,349.307,0.94003,02MAY23:12:28:56,265.417
4,HUN,348.0,34800120.0,34800006.0,08MS,034800,HUN01,3480000,34800.0,1.0,...,332.972,291.542,364.734,392.895,346.364,300.835,396.804,1.39470,02MAY23:12:28:54,377.679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6193,HUN,348.0,34800274.0,34808759.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,413.443,431.441,429.115,439.152,462.193,486.447,443.367,0.68586,02MAY23:12:28:55,400.780
6194,HUN,348.0,34800003.0,34808760.0,08MS,034800,HUN05,3480000,34800.0,1.0,...,339.807,420.217,381.860,410.052,379.095,405.908,386.596,0.59395,02MAY23:12:28:56,348.603
6195,HUN,348.0,34800209.0,34808761.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,446.405,368.627,441.561,430.339,369.407,452.986,392.082,0.72933,02MAY23:12:28:56,419.138
6196,HUN,348.0,34800125.0,34808764.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,393.713,286.782,303.800,436.778,413.684,368.423,365.617,0.97244,02MAY23:12:28:56,367.845


In [93]:
df_stu


,CNT,CNTRYID,CNTSCHID,CNTSTUID,CYC,NatCen,STRATUM,SUBNATIO,REGION,OECD,...,PV4MPRE,PV5MPRE,PV6MPRE,PV7MPRE,PV8MPRE,PV9MPRE,PV10MPRE,SENWT,VER_DAT,STUD_MATH
0,HUN,348.0,34800278.0,34800001.0,08MS,034800,HUN05,3480000,34800.0,1.0,...,640.519,611.919,645.604,625.981,598.866,617.845,588.401,0.89359,02MAY23:12:28:56,588.531
1,HUN,348.0,34800070.0,34800002.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,483.582,381.121,470.334,416.643,433.969,480.882,406.840,0.75935,02MAY23:12:28:55,397.170
2,HUN,348.0,34800201.0,34800003.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,618.097,686.393,672.865,690.185,677.799,739.687,701.354,0.76834,02MAY23:12:28:55,614.811
3,HUN,348.0,34800037.0,34800004.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,296.304,312.457,306.082,306.623,257.003,282.050,349.307,0.94003,02MAY23:12:28:56,265.417
4,HUN,348.0,34800120.0,34800006.0,08MS,034800,HUN01,3480000,34800.0,1.0,...,332.972,291.542,364.734,392.895,346.364,300.835,396.804,1.39470,02MAY23:12:28:54,377.679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6193,HUN,348.0,34800274.0,34808759.0,08MS,034800,HUN02,3480000,34800.0,1.0,...,413.443,431.441,429.115,439.152,462.193,486.447,443.367,0.68586,02MAY23:12:28:55,400.780
6194,HUN,348.0,34800003.0,34808760.0,08MS,034800,HUN05,3480000,34800.0,1.0,...,339.807,420.217,381.860,410.052,379.095,405.908,386.596,0.59395,02MAY23:12:28:56,348.603
6195,HUN,348.0,34800209.0,34808761.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,446.405,368.627,441.561,430.339,369.407,452.986,392.082,0.72933,02MAY23:12:28:56,419.138
6196,HUN,348.0,34800125.0,34808764.0,08MS,034800,HUN06,3480000,34800.0,1.0,...,393.713,286.782,303.800,436.778,413.684,368.423,365.617,0.97244,02MAY23:12:28:56,367.845
